In [ ]:
import h5py
import pandas
boxes = h5py.File(open('../data_cache/linked_boxes/epickitchens.h5','rb'))
TRAIN_LIST = "../epic-kitchens-100-annotations/EPIC_100_train.pkl"
train_file = pandas.read_pickle(TRAIN_LIST)
VAL_LIST = "../epic-kitchens-100-annotations/EPIC_100_validation.pkl"
val_file = pandas.read_pickle(VAL_LIST)
TEST_LIST = "../epic-kitchens-100-annotations/EPIC_100_test_timestamps.pkl"
test_file = pandas.read_pickle(TEST_LIST)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [ ]:
from tqdm import tqdm
total_frames = 0
overlap_hand_obj = 0
no_hand = 0
for input_file in [train_file,val_file,test_file]:
    for index, row in tqdm(input_file.iterrows()):
        video_id = row['video_id']
        start = row['start_frame']
        stop = row['stop_frame']
        for i in range(start, stop):
            
            try:
                num_hands = boxes[video_id][str(i)]['hand'].shape[0]
                num_objs = boxes[video_id][str(i)]['obj'].shape[0]    
            except:
                no_hand += 1
            for hand in range(num_hands):
                hand_box = boxes[video_id][str(i)]['hand'][hand, :][:4]
                
                for obj in range(num_objs):
                    obj_box = boxes[video_id][str(i)]['obj'][obj, :][:4]
                    iou = bb_intersection_over_union(hand_box, obj_box)
                    if iou > 0.5:
                        overlap_hand_obj += 1
                
           
            
            total_frames += 1

In [ ]:
hand_box